In [1]:
import pandas as pd 


train = pd.read_json('train.json')
test = pd.read_json('test.json')

In [2]:
train.head(3)

,Question,AnswerCount,Available Count,Score,is_accepted,ViewCount,Q_Id,Users Score,Answer,Q_Score,...,CreationDate,Title,Data Science and Machine Learning,Database and SQL,GUI and Desktop Applications,Networking and APIs,Other,Python Basics and Environment,System Administration and DevOps,Web Development
0,A researcher has created a small simulation in...,4,3,0.244919,False,2313,5136,5,One thing to remember is that the Matlab compi...,14,...,2008-08-07T18:47:00.000,Does anyone have experience creating a shared ...,1,0,1,0,0,0,0,0
1,A researcher has created a small simulation in...,4,3,0.099668,False,2313,5136,2,I'd also try ctypes first. \n\nUse the Matlab ...,14,...,2008-08-07T18:47:00.000,Does anyone have experience creating a shared ...,1,0,1,0,0,0,0,0
2,A researcher has created a small simulation in...,4,3,1.200000,True,2313,5136,3,I won't help much but I remember that I was ab...,14,...,2008-08-07T18:47:00.000,Does anyone have experience creating a shared ...,1,0,1,0,0,0,0,0


In [3]:
test.head(3)

,Question,AnswerCount,Available Count,Score,is_accepted,ViewCount,Q_Id,Users Score,Answer,Q_Score,...,CreationDate,Title,Data Science and Machine Learning,Database and SQL,GUI and Desktop Applications,Networking and APIs,Other,Python Basics and Environment,System Administration and DevOps,Web Development
0,I would like to reuse the same Spark applicati...,1,1,0.0,False,60,69312330,0,"To the best of my knowledge, it is not possibl...",2,...,2021-09-24T09:03:00.000,How to request the spark session object from a...,1,0,0,0,0,0,0,0
1,I am using pandas.read_csv() to read a csv fil...,1,1,0.0,False,42,69315906,0,Try to change the encoding of the file to UTF-...,0,...,2021-09-24T13:32:00.000,How to deal with special characters like � in ...,1,0,0,0,0,0,0,0
2,I am using pandas to read in csv data to my py...,2,1,0.0,False,33,69322728,0,That just means that one of the files has a ch...,0,...,2021-09-25T02:38:00.000,Encoding csv error with Pandas - have to encod...,1,0,0,0,0,0,0,0


In [4]:
df = pd.concat([train, test], sort=False)

In [5]:
output_df = df[['Question','Title', 'Answer']]
output_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7111 entries, 0 to 887
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Question  7111 non-null   object
 1   Title     7111 non-null   object
 2   Answer    7111 non-null   object
dtypes: object(3)
memory usage: 222.2+ KB


In [6]:
output_df.rename(columns={'Question': 'instruction', 'Title': 'context', 'Answer': 'response'}, inplace=True)


C:\Users\Admin\AppData\Local\Temp\ipykernel_21752\1948801372.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  output_df.rename(columns={'Question': 'instruction', 'Title': 'context', 'Answer': 'response'}, inplace=True)


In [7]:
# Save DataFrame to JSONL file
with open('output.jsonl', 'w') as f:
    for _, row in output_df.iterrows():
        json_line = row.to_json() + '\n'
        f.write(json_line)

## Resulted in 6% accuracy only

In [25]:
model_5_df = test[['Question', 'Answer']]
model_5_df.rename(columns={'Question': 'instruction', 'Answer': 'response'}, inplace=True)
model_5_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 888 entries, 0 to 887
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   instruction  888 non-null    object
 1   response     888 non-null    object
dtypes: object(2)
memory usage: 14.0+ KB


C:\Users\Admin\AppData\Local\Temp\ipykernel_21752\3936323419.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_5_df.rename(columns={'Question': 'instruction', 'Answer': 'response'}, inplace=True)


In [26]:
# find all duplicated instruction and keep only the longest response
model_5_df = model_5_df.sort_values('response', ascending=False).drop_duplicates('instruction')
model_5_df.reset_index(drop=True, inplace=True)

model_5_df['context'] = "" 


In [27]:
model_5_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 836 entries, 0 to 835
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   instruction  836 non-null    object
 1   response     836 non-null    object
 2   context      836 non-null    object
dtypes: object(3)
memory usage: 19.7+ KB


In [28]:
# Save DataFrame to JSONL file
with open('model5.jsonl', 'w') as f:
    for _, row in model_5_df.iterrows():
        json_line = row.to_json() + '\n'
        f.write(json_line)